### Analyzing Investment Portfolios for Ugandan Health Clinics Endownment
Investment Portfolio Project, ChemE 5660

______ ,_______ , Erik Kirakosyan

## Setup

In [34]:
include("Include.jl");

    Updating git-repo `https://github.com/varnerlab/VLQuantitativeFinancePackage.jl.git`


   Resolving package versions...


  No Changes to `~/Desktop/CHEME-5660-Project-Template-F23/Project.toml`
  No Changes to `~/Desktop/CHEME-5660-Project-Template-F23/Manifest.toml`


  Activating project at `~/Desktop/CHEME-5660-Project-Template-F23`


  No Changes to `~/Desktop/CHEME-5660-Project-Template-F23/Project.toml`
  No Changes to `~/Desktop/CHEME-5660-Project-Template-F23/Manifest.toml`


...

   Installed JuMP ─ v1.17.0


  No Changes to `~/Desktop/CHEME-5660-Project-Template-F23/Project.toml`
    Updating `~/Desktop/CHEME-5660-Project-Template-F23/Manifest.toml`
  [4076af6c] ↑ JuMP v1.16.0 ⇒ v1.17.0


Precompiling project...


  ✓ JuMP


  ✓ VLQuantitativeFinancePackage
  2 dependencies successfully precompiled in 21 seconds. 252 already precompiled.


## Prerequisites: Load historical dataset, compute expected returns and get 2023 `SPY` data
We gathered a daily open-high-low-close `dataset` for each firm in the [S&P500](https://en.wikipedia.org/wiki/S%26P_500) since `01-03-2018` until `12-01-2023`, along with data for a few exchange traded funds and volatility products during that time. 

In [37]:
original_dataset = load(joinpath(_PATH_TO_DATA, 
        "SP500-Daily-OHLC-1-3-2018-to-12-01-2023.jld2")) |> x-> x["dataset"];

### Clean the data
Not all of the tickers in our dataset have the maximum number of trading days for various reasons, e.g., acquistion or de-listing events. Let's collect only those tickers with the maximum number of trading days.

* First, let's compute the number of records for a company that we know has a maximim value, e.g., `AAPL` and save that value in the `maximum_number_trading_days` variable:

In [38]:
maximum_number_trading_days = original_dataset["AAPL"] |> nrow

1489

Now, lets iterate through our data and collect only those tickers that have `maximum_number_trading_days` records. Save that data in the `dataset::Dict{String,DataFrame}` variable:

In [39]:
dataset = Dict{String,DataFrame}();
for (ticker,data) ∈ original_dataset
    if (nrow(data) == maximum_number_trading_days)
        dataset[ticker] = data;
    end
end
dataset

Dict{String, DataFrame} with 459 entries:
  "NI"   => 1489×8 DataFrame…
  "EMR"  => 1489×8 DataFrame…
  "CTAS" => 1489×8 DataFrame…
  "HSIC" => 1489×8 DataFrame…
  "KIM"  => 1489×8 DataFrame…
  "PLD"  => 1489×8 DataFrame…
  "IEX"  => 1489×8 DataFrame…
  "BAC"  => 1489×8 DataFrame…
  "CBOE" => 1489×8 DataFrame…
  "EXR"  => 1489×8 DataFrame…
  "NCLH" => 1489×8 DataFrame…
  "CVS"  => 1489×8 DataFrame…
  "DRI"  => 1489×8 DataFrame…
  "DTE"  => 1489×8 DataFrame…
  "ZION" => 1489×8 DataFrame…
  "AVY"  => 1489×8 DataFrame…
  "EW"   => 1489×8 DataFrame…
  "EA"   => 1489×8 DataFrame…
  "NWSA" => 1489×8 DataFrame…
  ⋮      => ⋮

Let's get a list of firms that we have in cleaned up `dataset`, and save it in the `all_tickers` array:

In [40]:
all_tickers = keys(dataset) |> collect |> sort;
K = length(all_tickers);

### Get the 2023 `SPY` data

In [41]:
startdate = Date(2023,01,03);
SPY_dataset = dataset["SPY"];
SPY_df = filter(:timestamp => x-> x >= startdate, SPY_dataset);

### Compute the expected return for all firms in the dataset
The expected return $\mathbb{E}(r_{i})$ and covariance matrix $\Sigma$ will be used in our calculations, so we'll provide values for both of these items for the entire data set (all `N = 459` tickers), and then you can pick out which tickers you are interested in. 

* First, we compute the expected (annualized) log return by passing the `dataset` and the entire list of firms we have in the dataset (held in the $N\times{1}$ `all_array` array) to the `log_return_matrix(...)` method. The result is stored in the `all_firms_return_matrix` variable, a $T-1\times{N}$ array of log return values. Each row of `all_firms_return_matrix` corresponds to a time-value, while each column corresponds to a firm:

In [42]:
all_firms_return_matrix = log_return_matrix(dataset, all_tickers, 
    Δt = (1.0/252.0), risk_free_rate = 0.0);

## Your project starts here ....

Selected Portfolio Tickers: "NVDA", "MDT", "AMD", "PG", "MO", "PLD"

### Statement of Assumptions:

Similar to Lab 15b, we need to make assumptions to simplify our financial model. These assumptions are not completely realistic but greatly simplify the underlying math. 

1: The covariance matrix is constant over the period from 01-03-2018 to 12-01-2023. 

2: We are not taking into account short term capital gains tax, long term capital gains tax, restrictions on day trading and portfolio reallocation, or other market friction 

3: We are not taking into account real-world events and their impact on market volatility. 




These files are prerequisites for portfolio optimization. 

In [124]:
budget = 1000;
#REPLACE

1000

In [47]:
capital_allocation= load(joinpath(_PATH_TO_DATA,
        "CapitalAllocationLine-cesteam-PD1-CHEME-5660-Fall-2023.jld2")) |> x-> x["dataset"];
             
SIMs=load(joinpath(_PATH_TO_DATA, 
        "SIMs-cesteam-PD1-CHEME-5660-Fall-2023.jld2"))|> x-> x["sims"];
                

Dict{String, MySingleIndexModel} with 459 entries:
  "NI"   => MySingleIndexModel(-0.0457744, 0.66336, 0.05, Normal{Float64}(μ=-1.…
  "EMR"  => MySingleIndexModel(-0.0140694, 1.25195, 0.05, Normal{Float64}(μ=-2.…
  "CTAS" => MySingleIndexModel(0.137176, 1.20845, 0.05, Normal{Float64}(μ=9.786…
  "HSIC" => MySingleIndexModel(-0.0465561, 0.93735, 0.05, Normal{Float64}(μ=1.2…
  "KIM"  => MySingleIndexModel(-0.0426714, 1.28931, 0.05, Normal{Float64}(μ=-3.…
  "PLD"  => MySingleIndexModel(0.0444956, 1.00113, 0.05, Normal{Float64}(μ=6.64…
  "IEX"  => MySingleIndexModel(0.0405578, 0.98168, 0.05, Normal{Float64}(μ=3.02…
  "BAC"  => MySingleIndexModel(-0.0543229, 1.31835, 0.05, Normal{Float64}(μ=-1.…
  "CBOE" => MySingleIndexModel(-0.0581238, 0.624313, 0.05, Normal{Float64}(μ=1.…
  "EXR"  => MySingleIndexModel(0.0440579, 0.694857, 0.05, Normal{Float64}(μ=8.0…
  "NCLH" => MySingleIndexModel(-0.397087, 2.34477, 0.05, Normal{Float64}(μ=-2.1…
  "CVS"  => MySingleIndexModel(-0.01761, 0.797209, 0.05, N

In [43]:
efficient_frontier = load(joinpath(_PATH_TO_DATA,
        "EfficientFrontier-cesteam-PD1-CHEME-5660-Fall-2023.jld2")) |> x->x["dataset"]

Row,expected_excess_return,risk,tickers,w,risk_free_rate
,Float64,Float64,Array…,Array…,Float64
1,1.43946e-7,0.15205,"[""NVDA"", ""MDT"", ""AMD"", ""PG"", ""MO"", ""PLD""]","[1.25973e-7, 0.163492, 2.041e-7, 0.526952, 0.182744, 0.126812]",0.05
2,0.00454553,0.152389,"[""NVDA"", ""MDT"", ""AMD"", ""PG"", ""MO"", ""PLD""]","[1.43311e-7, 0.152205, 3.11878e-7, 0.541592, 0.166631, 0.139571]",0.05
3,0.0090909,0.152906,"[""NVDA"", ""MDT"", ""AMD"", ""PG"", ""MO"", ""PLD""]","[-3.00804e-9, 0.140918, 1.50331e-8, 0.556233, 0.150519, 0.15233]",0.05
4,0.0136364,0.153602,"[""NVDA"", ""MDT"", ""AMD"", ""PG"", ""MO"", ""PLD""]","[-1.79631e-9, 0.129631, 4.45449e-7, 0.570874, 0.134406, 0.165089]",0.05
5,0.0181818,0.154445,"[""NVDA"", ""MDT"", ""AMD"", ""PG"", ""MO"", ""PLD""]","[2.12656e-7, 0.120293, 0.00527865, 0.579719, 0.123312, 0.171397]",0.05
6,0.0227273,0.155387,"[""NVDA"", ""MDT"", ""AMD"", ""PG"", ""MO"", ""PLD""]","[2.38957e-7, 0.111319, 0.0115415, 0.587484, 0.113155, 0.176501]",0.05
7,0.0272728,0.156426,"[""NVDA"", ""MDT"", ""AMD"", ""PG"", ""MO"", ""PLD""]","[2.55663e-7, 0.102345, 0.0178048, 0.595248, 0.102998, 0.181604]",0.05
8,0.0318182,0.157558,"[""NVDA"", ""MDT"", ""AMD"", ""PG"", ""MO"", ""PLD""]","[2.86795e-7, 0.0933715, 0.024068, 0.603012, 0.0928404, 0.186708]",0.05
9,0.0363637,0.158782,"[""NVDA"", ""MDT"", ""AMD"", ""PG"", ""MO"", ""PLD""]","[3.39804e-7, 0.0843978, 0.0303313, 0.610776, 0.0826833, 0.191812]",0.05


### Task One: Initial Allocation

In [49]:
Find the portfolio with the maximum expected excess returns

findmax(efficient_frontier[!,"expected_excess_return"])

(0.2863636276792971, 64)

We will use Portfolio 64. Using results from Lab 15b, we can find what is in this portfolio by sampling the array generated above 

In [58]:
portfolio=64;
expected_excess_return=efficient_frontier[portfolio, :expected_excess_return]
risk=efficient_frontier[portfolio, :risk]
tickers=efficient_frontier[portfolio, :tickers]
w=efficient_frontier[portfolio, :w]
risk_free_rate=efficient_frontier[portfolio, :risk_free_rate]

println("Portfolio #$(portfolio) has an exected excess return of $(expected_excess_return) with risk = $(risk). 
The risk free rate is $(risk_free_rate)%.")

Portfolio #64 has an exected excess return of 0.2863636276792971 with risk = 0.4594230792505488. 
The risk free rate is 0.05%.


In [91]:
w[2]


-2.4994641898163153e-9

In [105]:
i=0
for item in efficient_frontier[portfolio, :tickers]
    i+=1
    if w[i]<0.00001
        weight=0 
    else
        weight=w[i]*100
    end
    println("$item forms $weight% of our portfolio")
end

NVDA forms 3.2421779736775127% of our portfolio
MDT forms 0% of our portfolio
AMD forms 96.75781803071864% of our portfolio
PG forms 0% of our portfolio
MO forms 0% of our portfolio
PLD forms 0% of our portfolio


We now need to know how many shares we purchased of each stock

In [131]:
# From Lab15b
price_initial=Array{Float64,1}();
for stock in tickers
    price_df=dataset[stock]
    price=filter(:timestamp=> y-> y>= startdate, price_df) |> y-> y[1,:close]
    push!(price_initial, price)
end


In [132]:
i=0
weight_initial=Array{Float64,1}();
for item in efficient_frontier[portfolio, :tickers]
    i+=1
    if w[i]<0.00001
        weight=0 
        push!(weight_initial, weight)
    else
        weight=w[i]
    push!(weight_initial, weight)
    end
end


In [133]:
tickers_df = DataFrame();
m=0
for i in efficient_frontier[portfolio, :tickers]
    m+=1
    allocate=budget*weight_initial[m]
    shares=allocate/price_initial[m]
    println("For ticker $i, we will allocate $allocate to purchase $shares shares")
end
    

For ticker NVDA, we will allocate 32.42177973677512 to purchase 0.22648815743468473 shares
For ticker MDT, we will allocate 0.0 to purchase 0.0 shares
For ticker AMD, we will allocate 967.5781803071864 to purchase 15.113686040412158 shares
For ticker PG, we will allocate 0.0 to purchase 0.0 shares
For ticker MO, we will allocate 0.0 to purchase 0.0 shares
For ticker PLD, we will allocate 0.0 to purchase 0.0 shares


### Task Two: Reallocation after t=0

In [ ]:
trading_days= 229 # Difference between Jan 3 and Dec 1 of 2023, with weekends and holidays removed

## Disclaimer and Risks
__This content is offered solely for training and  informational purposes__. No offer or solicitation to buy or sell securities or derivative products, or any investment or trading advice or strategy,  is made, given, or endorsed by the teaching team. 

__Trading involves risk__. Carefully review your financial situation before investing in securities, futures contracts, options, or commodity interests. Past performance, whether actual or indicated by historical tests of strategies, is no guarantee of future performance or success. Trading is generally inappropriate for someone with limited resources, investment or trading experience, or a low-risk tolerance.  Only risk capital that is not required for living expenses.

__You are fully responsible for any investment or trading decisions you make__. Such decisions should be based solely on your evaluation of your financial circumstances, investment or trading objectives, risk tolerance, and liquidity needs.